## 크롤링 물품 리스트 정리 자동화 프로그램
### 요청 사항
- 물품 품목이 같은 것들을 묶어서 최대 200개까지 작성
- 카테고리 중 다이소 매장 물품인 품목을 우선적으로 선별
- 물품은 가격 순으로 정렬
- 상품 사진 추가
- 10만 원 이하의 물품만 선정


In [ ]:
# 크롤링 엑셀 정리 코드

from openpyxl import Workbook
import pandas as pd
from io import BytesIO
from openpyxl.drawing.image import Image
import requests
import numpy as np

FILE_NAME = 'C:\\Users\\user\\Desktop\\크롤링 결과\\정보저장매체\\촬영 대상 물품 분류체계_v0.1_권혁진_다이소몰 크롤링 결과_정보저장매체_텍스.xlsx'

wb = Workbook()
ws = wb.active
df = pd.read_excel(FILE_NAME, header=None)

ws.append(['물품분류', '물품코드', '물품종', '순번', '상품번호', '카테고리', '상품명', '상품사진', '가격', '링크'])
ws.column_dimensions['H'].width = 50

def init():
    result = list(np.unique(df[0]))
    result.remove('물품분류')

    print(result)
    print("----------------------")
    start = 2 # 셀 높이 조정할 때 사용하는 변수

    for name in result:
        stuff = df.loc[df[0] == name]

        stuff_1 = stuff[stuff[5].str.contains('다이소')]
        stuff_2 = stuff[~stuff[5].str.contains('다이소')]

        # 정렬하는 플로우
        stuff_1 = stuff_1.sort_values(by=8, ascending=True)
        stuff_2 = stuff_2.sort_values(by=8, ascending=True)

        data = pd.concat([stuff_1, stuff_2])

        data = data.loc[data[8] < 100000]

        data = data.index.to_list()
        print(name)
        print(len(data))
        print("---------------------------")

        if (len(data) > 200):
            data = data[:200] # 200 개까지만 자르기

        start = insert(start, data) # 엑셀에 집어 넣기

    wb.save('C:\\Users\\user\\Desktop\\result.xlsx')

def image(url, no): # 이미지 삽입 함수
    res = requests.get(url)
    image_data = BytesIO(res.content)
    img = Image(image_data)
    ws.add_image(img, anchor='H%s' %no)

def insert(start, data): # data -> index 가 담긴 리스트
    j = start
    for i in data:
        ws.row_dimensions[j].height = 200 # 셀의 높이 조정
        image(df[7][i], j)
        ws.append([df[0][i], df[1][i], df[2][i], df[3][i], df[4][i], df[5][i], df[6][i], '', df[8][i], df[9][i]])
        j = j + 1
    return j

init()

In [ ]:
# 소분류 물품 코드와 물품명 일치 확인 후 변경 코드
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import os

def change(ws, rows, status):
    for row in rows:
        ws.cell(row=row+1, column=2).value = status

# 참고 파일명
FILE_NAME = 'C:\\Users\\user\\Desktop\\촬영 대상 물품 분류체계_v0.6_이영교.xlsx'

# 0 -> 소분류 code, 1-> 이름
df = pd.read_excel(FILE_NAME, header=None, sheet_name=1, usecols=[0, 1])

FOLDER = 'C:\\Users\\user\\Desktop\\웹크롤링 결과_구매 물품 정리_v0.2'

for (root, dirs, files) in os.walk(FOLDER):
    for file in files:
        file_path = os.path.join(root, file)

        print(file_path)

        # 0 -> 이름, 1 -> code
        df1 = pd.read_excel(file_path, header=None, usecols=[0, 1])

        # 수정할 파일
        wb = load_workbook(file_path)
        ws = wb.active
    
        result = list(np.unique(df1[0]))
        result.remove('물품분류')
        print(result)

        for name in result:
            status = df.loc[df[1] == name][0].values[0]
            status_compare = df1.loc[df1[0] == name][1].values[0]

            print(name, '| 실제 : ', status, ', 크롤링 Code : ', status_compare)

            if (status == status_compare):
                print('ok\n-------------------')
                continue
            
            data = df1.index[df1[0] == name].tolist()
            print("change 시작 포인트: %s\n--------------------" %data[0])

            change(ws, data, status)

        wb.save(os.path.join(root, '%s.xlsx' %root.split("\\")[-1]))

In [ ]:
# 물품 종 별 카운트, 총액, 평균 금액
import pandas as pd
import numpy as np
from openpyxl import Workbook

wb = Workbook()
ws = wb.active
ws.append(['물품분류', '물품 Code', 'Count', 'Avg', 'Sum'])

FOLDER = 'C:\\Users\\user\\Desktop\\웹크롤링 결과_구매 물품 정리_v0.2'

for (root, dirs, files) in os.walk(FOLDER):
    for file in files:
        file_path = os.path.join(root, file)

        df = pd.read_excel(file_path, header=None, usecols=[0, 1, 8])
        print(file_path)

        result = list(np.unique(df[0]))
        result.remove('물품분류')

        for name in result:
            data = df.index[df[0] == name].tolist()
            status = df.loc[df[0] == name][1].values[0]

            sum = 0
            count = 0

            for i in data:
                count = count + 1
                sum = sum + df[8][i]

            ws.append([name, status, count, sum//count, sum])

wb.save('C:\\Users\\user\\Desktop\\웹크롤링 결과_구매 물품 정리_v0.2\\물품 별 통계.xlsx')
        